# Development grounds for code

In [38]:
from sklearn.svm import LinearSVC
from vehicle_detector import *
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from keras.models import Sequential
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, SpatialDropout2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

import cv2

%matplotlib inline

In [39]:
def get_data(sample_size=18458):    
    notcar_data_folders = ['./data/non-vehicles/Extras',
                           './data/non-vehicles/GTI']

    car_data_folders    = ['./data/vehicles/GTI_MiddleClose',
                            './data/vehicles/GTI_Far',
                            './data/vehicles/KITTI_extracted',
                            './data/vehicles/GTI_Right',
                            './data/vehicles/GTI_Left']

    cars = []
    notcars = []
    y_one_hot = []
    for folder in notcar_data_folders:
        image_paths =glob.glob(folder+'/*')
        for path in image_paths:
            notcars.append(cv2.imread(path))
            # One_hot_label encoding
            y_one_hot.append([0,1])
            if len(notcars) == sample_size:
                break
        if len(notcars) == sample_size:
                break


    for folder in car_data_folders:
        image_paths =glob.glob(folder+'/*')
        for path in image_paths:
            cars.append(cv2.imread(path))
            y_one_hot.append([1,0])
            # One_hot_label encoding
            if len(cars) == sample_size:
                break
        if len(cars) == sample_size:
                break


    cars = np.array(cars)
    notcars = np.array(notcars)
    y_one_hot = np.array(y_one_hot)

    X = np.concatenate((cars, notcars), axis=0)

    del cars
    del notcars
    
    with open('./data/data.p', 'wb') as f:
        pickle.dump((X, y_one_hot), f, pickle.HIGHEST_PROTOCOL)

# Uncomment for getting new samples sizes        
get_data(sample_size=1000)

with open('./data/data.p', 'rb') as f:
    X, y_one_hot = pickle.load(f)
print('loaded..')


loaded..


In [40]:


def resize_to_target_size(image):
    TARGET_SIZE = (64,64)
    return cv2.resize(image, TARGET_SIZE)

def preprocess_image(image):
    image = resize_to_target_size(image)
    image = image.astype(np.float32)
    # Normalize image
    image = image / 255.0 - 0.5
    return image

X_normalized = []

for img in X:
    X_normalized.append(preprocess_image(img))
    
X_normalized = np.array(X_normalized)

X_train, X_test, y_train, y_test = train_test_split(X_normalized,
                                                    y_one_hot,
                                                    random_state=42)


In [41]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

# TODO: Compile and train the model
model.compile('adam', 'binary_crossentropy', ['accuracy'])
model.fit(X_train,
          y_train,
          batch_size=32,
          nb_epoch=5,
          verbose=1,
          validation_split=0.1,
          shuffle= True)


Train on 1350 samples, validate on 150 samples
Epoch 1/5
1350/1350 [==============================] - 19s - loss: 0.3315 - acc: 0.8548 - val_loss: 0.1238 - val_acc: 0.9467
Epoch 2/5
1350/1350 [==============================] - 19s - loss: 0.0908 - acc: 0.9704 - val_loss: 0.0518 - val_acc: 1.0000
Epoch 3/5
1350/1350 [==============================] - 17s - loss: 0.0359 - acc: 0.9889 - val_loss: 0.0315 - val_acc: 0.9933
Epoch 4/5
1350/1350 [==============================] - 17s - loss: 0.0198 - acc: 0.9985 - val_loss: 0.0294 - val_acc: 0.9867
Epoch 5/5
1350/1350 [==============================] - 16s - loss: 0.0180 - acc: 0.9933 - val_loss: 0.0160 - val_acc: 1.0000


In [42]:
model.save('./models/keras.h5')

In [ ]:
# TODO: Evaluate model on test data

metrics = model.evaluate(X_test, y_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))